# Exploratory Data Analysis

## Import libraries

In [135]:
import pandas as pd

## Read in excel file to pandas

In [136]:
api_df = pd.read_excel("../data/raw/RiskClassification_Data_Endpoints_V2.xlsx", "Core_Endpoint",
                      usecols = "A:R")
# rename column "security_test_result (FALSE=Passed; TRUE=Failed)" to "security_test_result"
api_df.rename(columns={'security_test_result (FALSE=Passed; TRUE=Failed)': 'security_test_result'}, inplace=True)
# get only api_id, parameters and response_metadata columns
api_df = api_df[['api_id', 'parameters', 'response_metadata']]
api_df.head()


,api_id,parameters,response_metadata
0,1117,"{""q"": ""Running""}",NaN
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v..."
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser..."
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser..."
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var..."


# Data wrangling
## Extract information from metadata

Metadata contains rich information about the API. It is not too difficult to extract information from the metadata since it is a JSON object. However, each API has different metadata. Therefore, we need to extract the key fields from the metadata and use it to create new columns in the dataframe.

In [137]:
# get a list of all response_metadata
metadata_list = api_df['response_metadata'].tolist()
# get a list of all parameters
parameters_list = api_df['parameters'].tolist()
# replace nan with empty string
metadata_list = [str(x) if pd.notnull(x) else '{}' for x in metadata_list]
parameters_list = [str(x) if pd.notnull(x) else '{}' for x in parameters_list]

key_set = set()  # make sure there is no duplicate key
# loop through each response_metadata and extract key value
for i in range(len(metadata_list)):
    if metadata_list[i] is not None:
        # convert metadata_list[i] to dictionary
        metadata_dict = eval(metadata_list[i])
        # loop through metadata_dict and extract key value
        for key, value in metadata_dict.items():
            key_set.add(key.lower())

print(f'Total number of fields in response_metadata: {len(key_set)}')


Total number of fields in response_metadata: 127


### Selecting fields for feature engineering
We got 127 new fields, that's quite a lot. It's obvious not all fields are needed. We will use the header security list from OWASP to cross-reference the fields in the metadata: 
https://cheatsheetseries.owasp.org/cheatsheets/HTTP_Headers_Cheat_Sheet.html

The following list contains the fields appear in both our key_set and OWASP_list:

In [138]:
# list of columns that have high risk security
high_risk_security_headers = [
    'x-frame-options',
    'x-xss-protection',
    'strict-transport-security',
    'expect-ct',
    'referrer-policy',
    'content-type',
    'set-cookie',
    'access-control-allow-origin',
    'server',
    'x-powered-by',
    'x-aspnet-version',
    'x-ratelimit-limit'
]


Once we know the fields we need, we can create a new columns containing the information and obtain the new dataframe. 

In [139]:
metadata_count_df = pd.DataFrame(columns=['metadata_fields_count'])
parameters_count_df = pd.DataFrame(columns=['parameters_count'])
for i in range(len(metadata_list)):
    metadata_fields_count = 0  # keep track of how many fields in each API
    parameters_count = 0  # keep track of how many parameters in each API
    if metadata_list[i] is not None:
        metadata_dict = eval(metadata_list[i])
        # loop through metadata_dict and extract key value
        for key, value in metadata_dict.items():
            key = key.lower()
            if key in high_risk_security_headers:
                api_df.loc[i, key] = value
                metadata_fields_count += 1
    metadata_count_df.loc[i, 'metadata_fields_count'] = metadata_fields_count
    # repeat for parameters
    if parameters_list[i] is not None:
        parameters_dict = eval(parameters_list[i])
        parameters_count = len(parameters_dict)
    parameters_count_df.loc[i, 'parameters_count'] = parameters_count
api_df = api_df.assign(metadata_fields_count=metadata_count_df)
api_df = api_df.assign(parameters_count=parameters_count_df)
api_df



,api_id,parameters,response_metadata,server,set-cookie,content-type,referrer-policy,x-frame-options,x-xss-protection,strict-transport-security,expect-ct,x-ratelimit-limit,x-powered-by,x-aspnet-version,access-control-allow-origin,metadata_fields_count,parameters_count
0,1117,"{""q"": ""Running""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v...",istio-envoy,incap_ses_998_2087933=bQUEEEwrgDTdNnYX0JzZDXyP...,application/json,no-referrer,"DENY, DENY","1; mode=block, 1 ; mode=block",max-age=31557600;includeSubDomains,NaN,NaN,NaN,NaN,NaN,7,0
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser...",AkamaiGHost,NaN,text/html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser...",nginx/1.6.2,NaN,application/json; charset=utf-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var...",ESF,NaN,application/json; charset=UTF-8,NaN,SAMEORIGIN,0,NaN,NaN,NaN,NaN,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1132,{},"{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,application/json,same-origin,DENY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
109,1132,"{""page"": ""2""}","{""Age"": ""3280"", ""NEL"": ""{\""success_fraction\"":...",cloudflare,NaN,application/json; charset=utf-8,NaN,NaN,NaN,NaN,"max-age=604800, report-uri=""https://report-uri...",NaN,Express,NaN,*,5,1
110,1286,"{""Content-Disposition"": ""attachment; filename=...","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,application/json,same-origin,DENY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
111,1286,"{""intParam"": 1}","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,application/json,same-origin,DENY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1


## Feature engineering
For all the new fields, we will convert the string value to the corresponding numerical value.

In [140]:
# recommend rule for each high risk security header
# to make it simple, if the header is not present, it is considered as not secure
x_frame_options = 'DENY'
x_xss_protection = '0'
content_security_policy = '0'
strict_transport_security = 'includeSubDomains'
expect_ct = 'max-age'
referrer_policy = 'strict-origin-when-cross-origin'
content_type = 'charset'
set_cookie = 'Secure'
access_control_allow_origin = 'https'
# make a dictionary to store the recommended rule
recommend_dict = {
    'x-frame-options': x_frame_options,
    'x-xss-protection': x_xss_protection,
    'strict-transport-security': strict_transport_security,
    'expect-ct': expect_ct,
    'referrer-policy': referrer_policy,
    'content-type': content_type,
    'set-cookie': set_cookie,
    'access-control-allow-origin': access_control_allow_origin
}

# the following headers are not recommended
should_not_be_present = ['server',
                         'x-powered-by',
                         'x-aspnet-version']
good_to_be_present = ['x-ratelimit-limit']                         


### Assign value for recommend_dict
To make it simple, we will use only binary value for recommend_dict, 0: low risk and 1: high risk.

In [141]:
# loop through each row of api_df
for i in range(len(api_df)):
    # loop through recommend_dict
    for key, value in recommend_dict.items():
        # check if the cell is not NaN
        if not pd.isna(api_df.loc[i, key]):
            # if the cell contain the value in recommend_dict, then assign the value 0, else assign 1
            api_df.loc[i, key] = 0 if value in api_df.loc[i, key] else 1
api_df

,api_id,parameters,response_metadata,server,set-cookie,content-type,referrer-policy,x-frame-options,x-xss-protection,strict-transport-security,expect-ct,x-ratelimit-limit,x-powered-by,x-aspnet-version,access-control-allow-origin,metadata_fields_count,parameters_count
0,1117,"{""q"": ""Running""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v...",istio-envoy,0,1,1,0,1,0,NaN,NaN,NaN,NaN,NaN,7,0
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser...",AkamaiGHost,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser...",nginx/1.6.2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var...",ESF,NaN,0,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1132,{},"{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
109,1132,"{""page"": ""2""}","{""Age"": ""3280"", ""NEL"": ""{\""success_fraction\"":...",cloudflare,NaN,0,NaN,NaN,NaN,NaN,0,NaN,Express,NaN,1,5,1
110,1286,"{""Content-Disposition"": ""attachment; filename=...","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
111,1286,"{""intParam"": 1}","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",gunicorn,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1


### Assign value for should_not_be_present
Set 1 if the field present in the header.

In [142]:
# loop through each row of api_df
for i in range(len(api_df)):
    # loop through should_not_be_present
    for field in should_not_be_present:
        # check if the cell is not NaN
        if not pd.isna(api_df.loc[i, field]):
            # if the cell contain value, then assign the value 1
            api_df.loc[i, field] = 1
api_df


,api_id,parameters,response_metadata,server,set-cookie,content-type,referrer-policy,x-frame-options,x-xss-protection,strict-transport-security,expect-ct,x-ratelimit-limit,x-powered-by,x-aspnet-version,access-control-allow-origin,metadata_fields_count,parameters_count
0,1117,"{""q"": ""Running""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v...",1,0,1,1,0,1,0,NaN,NaN,NaN,NaN,NaN,7,0
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser...",1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser...",1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var...",1,NaN,0,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1132,{},"{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
109,1132,"{""page"": ""2""}","{""Age"": ""3280"", ""NEL"": ""{\""success_fraction\"":...",1,NaN,0,NaN,NaN,NaN,NaN,0,NaN,1,NaN,1,5,1
110,1286,"{""Content-Disposition"": ""attachment; filename=...","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
111,1286,"{""intParam"": 1}","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1


### Assign value for should_be_present
Set 0 if the field present in the header.

In [143]:
# loop through each row of api_df
for i in range(len(api_df)):
    # loop through good_to_be_present
    for field in good_to_be_present:
        # check if the cell is not NaN
        if not pd.isna(api_df.loc[i, field]):
            # if the cell contain value, then assign the value 0
            api_df.loc[i, field] = 0
api_df


,api_id,parameters,response_metadata,server,set-cookie,content-type,referrer-policy,x-frame-options,x-xss-protection,strict-transport-security,expect-ct,x-ratelimit-limit,x-powered-by,x-aspnet-version,access-control-allow-origin,metadata_fields_count,parameters_count
0,1117,"{""q"": ""Running""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v...",1,0,1,1,0,1,0,NaN,NaN,NaN,NaN,NaN,7,0
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser...",1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser...",1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var...",1,NaN,0,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1132,{},"{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
109,1132,"{""page"": ""2""}","{""Age"": ""3280"", ""NEL"": ""{\""success_fraction\"":...",1,NaN,0,NaN,NaN,NaN,NaN,0,NaN,1,NaN,1,5,1
110,1286,"{""Content-Disposition"": ""attachment; filename=...","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
111,1286,"{""intParam"": 1}","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,NaN,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1


### Imputation for NaN values
Since there are a lot of NaN fields, we will fill them with the most common value which is the "low risk" option.

In [144]:
# replace NaN value with 0
api_df = api_df.fillna(0)
api_df


,api_id,parameters,response_metadata,server,set-cookie,content-type,referrer-policy,x-frame-options,x-xss-protection,strict-transport-security,expect-ct,x-ratelimit-limit,x-powered-by,x-aspnet-version,access-control-allow-origin,metadata_fields_count,parameters_count
0,1117,"{""q"": ""Running""}",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1148,{},"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v...",1,0,1,1,0,1,0,0,0,0,0,0,7,0
2,1147,"{""v"": ""1""}","{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser...",1,0,1,0,0,0,0,0,0,0,0,0,2,1
3,1119,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...","{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser...",1,0,0,0,0,0,0,0,0,0,0,0,2,3
4,1050,"{""q"": ""Dehri, Bihar, India""}","{""Date"": ""Thu, 07 Oct 2021 19:14:31 GMT"", ""Var...",1,0,0,0,1,0,0,0,0,0,0,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1132,{},"{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,0,1,1,0,0,0,0,0,0,0,0,4,0
109,1132,"{""page"": ""2""}","{""Age"": ""3280"", ""NEL"": ""{\""success_fraction\"":...",1,0,0,0,0,0,0,0,0,1,0,1,5,1
110,1286,"{""Content-Disposition"": ""attachment; filename=...","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,0,1,1,0,0,0,0,0,0,0,0,4,1
111,1286,"{""intParam"": 1}","{""Via"": ""1.1 vegur"", ""Date"": ""Tue, 08 Mar 2022...",1,0,1,1,0,0,0,0,0,0,0,0,4,1


### Export data to excel

In [145]:
# save api_df to excel
api_df.to_excel("../data/processed/api_df_count.xlsx", index=False)